# Data Upload
---

**Imports**

In [1]:
import sshtunnel
import mysql.connector
from mysql.connector import errorcode

import pandas as pd

**SSH settings**

In [2]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

**Table Schema**

In [3]:
TABLES = {}

TABLES['sightings'] = (
    "CREATE TABLE `sightings` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `lat` Decimal(9,6) NOT NULL,"
    "  `long` Decimal(9,6) NOT NULL,"
    "  `timestamp` varchar(30) NOT NULL,"
    "  `species` varchar(50) NOT NULL,"
    "  `pic_url` varchar(500) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ")")

TABLES['research'] = (
    "CREATE TABLE `research` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `lat` Decimal(9,6) NOT NULL,"
    "  `long` Decimal(9,6) NOT NULL,"
    "  `timestamp` varchar(30) NOT NULL,"
    "  `species` varchar(50) NOT NULL,"
    "  `subject_id` varchar(50),"
    "  PRIMARY KEY (`id`)"
    ")")

TABLES['corridors'] = (
    "CREATE TABLE `corridors` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `lat` date NOT NULL,"
    "  `long` varchar(14) NOT NULL,"
    "  `vert_spread` varchar(16) NOT NULL,"
    "  `horiz_spread` varchar(16) NOT NULL,"
    "  `pic_url` enum('M','F') NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ")")

**Helper Functions**

In [4]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
    
    return True
        
def upload_query_gen(file_name, record):
    return ("INSERT INTO {} VALUES (" + "{}, "*3 + "'{}' ,"*(len(record)-4) + "'{}')").format(file_name, *list(record))

**SSH Tunnel**

In [5]:
tunnel =  sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='rangerradar', 
    ssh_password='fv1s2r112!@',
    remote_bind_address=('rangerradar.mysql.pythonanywhere-services.com', 3306)
)

tunnel.start()

**MySQL Connection**

In [6]:
connection = mysql.connector.connect(
    user='rangerradar', 
    password='rrsql12!@',
    host='rangerradar.mysql.pythonanywhere-services.com', 
    port=3306,
    database='rangerradar$sdzoohack',
)
    
cursor = connection.cursor()
DB_NAME = 'sdzoohack'

**Create Tables Using Schema Dict**

In [7]:
for table_name in TABLES:
    
    table_description = TABLES[table_name]
    
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    
    else:
        print("done.")

Creating table sightings: done.
Creating table research: done.
Creating table corridors: done.


**Iterate through cleaned CSVs and push to DB**

In [8]:
for file_name in ['sightings', 'research']:
    
    print(f'Working on {file_name}...')
    
    try:
        df = pd.read_csv(f'../data/{file_name}_to_sql.csv').reset_index()
        # print(df.shape)
    
    except:
        continue

    for i, record in enumerate(df.values):
        
        if i % 500 == 0:
            connection.commit()
            print(f'{(i / df.shape[0]) * 100} percent done')
        
        query = upload_query_gen(file_name, record)
        # print(query)
        
        try:
            cursor.execute(query)
    
        except mysql.connector.Error as err:
            print(err)
            continue
    
    print(f'{file_name} finished.')

print('----------\nDone.')

Working on sightings...
0.0 percent done
1062 (23000): Duplicate entry '1' for key 'PRIMARY'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
14.068655036578503 percent done
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42

84.01008120974517 percent done
84.51014121694604 percent done
85.0102012241469 percent done
85.51026123134776 percent done
86.01032123854863 percent done
86.51038124574949 percent done
87.01044125295036 percent done
87.51050126015122 percent done
88.01056126735209 percent done
88.51062127455295 percent done
89.01068128175382 percent done
89.51074128895468 percent done
90.01080129615553 percent done
90.5108613033564 percent done
91.01092131055726 percent done
91.51098131775814 percent done
92.011041324959 percent done
92.51110133215987 percent done
93.01116133936073 percent done
93.51122134656158 percent done
94.01128135376246 percent done
94.51134136096331 percent done
95.01140136816419 percent done
95.51146137536504 percent done
96.01152138256592 percent done
96.51158138976677 percent done
97.01164139696765 percent done
97.5117014041685 percent done
98.01176141136936 percent done
98.51182141857024 percent done
99.0118814257711 percent done
99.51194143297197 percent done
research finis

**Close Connections**

In [11]:
cursor.close()
connection.close()